In [ ]:
from utils.db import get_connection
from dotenv import load_dotenv
import utils.links as links
import pandas as pd
import requests
import os

load_dotenv()

try:
    api_key = os.getenv("API_KEY") 

    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }

    conn = get_connection()
    cursor = conn.cursor()
    print("Connection Successful")
except Exception as e:
    print("Connection Failed: ", e)

<h1>Season Table</h1>

In [ ]:
# Creation & Insertion of seasons table
try:
    season_res = requests.get(links.seasons_url,headers=headers)
    season_res = season_res.json()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS seasons (
        season_id CHAR(36) PRIMARY KEY,
        year INT NOT NULL,
        start_date DATE NOT NULL,
        end_date DATE NOT NULL,
        status VARCHAR(20) NOT NULL,
        type_code VARCHAR(10) NOT NULL
    );
    """)


    # Insert Query
    insert_query = "INSERT INTO seasons (season_id, year, start_date, end_date, status, type_code) VALUES  (%s, %s, %s, %s, %s, %s);"

    # Combining rows into a list
    season_rows = [
        (row["id"],row["year"],row["start_date"],row["end_date"],row["status"],row["type"]["code"])
        for row in season_res["seasons"]
    ]
    
    cursor.executemany(insert_query,season_rows)
    conn.commit()

    print("Insertion in Season Table Successful")
except Exception as e:
    print("Error Occurred: ", e)

<h1>Venues Table</h1>

In [ ]:
# Creation & Insertion of venues table
try:
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS venues (
            venue_id CHAR(36) PRIMARY KEY,
            name VARCHAR(50) NOT NULL,
            city VARCHAR(25) NOT NULL,
            state VARCHAR(10) NOT NULL,
            country VARCHAR(10) NOT NULL,
            zip VARCHAR(10) NOT NULL,
            address VARCHAR(50) NOT NULL,
            capacity INT NOT NULL,
            surface VARCHAR(25) NOT NULL,
            roof_type VARCHAR(25) NOT NULL,
            latitude FLOAT NOT NULL,
            longitude FLOAT NOT NULL
        );
    """)
    
    # Insert Query
    insert_query = "INSERT IGNORE INTO venues (venue_id, name, city, state, country, zip, address, capacity, surface, roof_type, latitude, longitude) VALUES  (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"

    cursor.execute("SELECT year FROM seasons ORDER BY year DESC;")
    season_years = [r[0] for r in cursor.fetchall()]


    for season_year in season_years:
        venues_res = requests.get(links.venues_url.format(season_year=season_year),headers=headers)
        venues_res = venues_res.json()
        for week in venues_res["weeks"]:
            for game in week["games"]:
                venue = game.get("venue", {})
                if not venue:
                    continue

                id = venue.get("id")
                name = venue.get("name")
                city = venue.get("city")
                state = venue.get("state")
                country = venue.get("country")
                zip = venue.get("zip")
                address = venue.get("address")
                capacity = venue.get("capacity")
                surface = venue.get("surface")
                roof_type = venue.get("roof_type")
                latitude = venue.get("location", {}).get("lat")
                longitude = venue.get("location", {}).get("lng")

                if not all([id,name,city,state,country,zip,address,capacity,surface,roof_type,latitude,longitude]):
                    continue

                cursor.execute(insert_query, (id,name,city,state,country,zip,address,capacity,surface,roof_type,latitude,longitude))

    conn.commit()

    print("Insertion in Venue Table Successful")
except Exception as e:
    print("Error Occurred: ", e)

In [ ]:
cursor.close()
conn.close()